In [ ]:
# 检测 chat_rag.py 文件
file_path = "../app.py"
# file_path = "../Module/Components/state_manager.py"
# file_path = "../Module/Components/config.py"
!flake8 {file_path} --max-line-length=240
!pylint {file_path}

In [ ]:
%%writefile ..\app_temp.py
# pylint: disable=import-error  # Project structure requires dynamic path handling
# pylint: disable=wrong-import-position  # Path setup must come before local imports
"""
For more information on `huggingface_hub` Inference API support
please check the docs: https://huggingface.co/docs/huggingface_hub/v0.22.2/en/guides/inference
"""
import os
import sys
import json
from typing import Dict, Any, List, Tuple
import gradio as gr
from google import genai
# ===== 2. 初始化配置 =====
# 获取当前文件所在目录的绝对路径
if "__file__" in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
    root_dir = os.path.normpath(os.path.join(current_dir, ".."))
else:
    # 在 Jupyter Notebook 环境中
    current_dir = os.getcwd()
    current_dir = os.path.join(current_dir, "..")
    root_dir = os.path.normpath(os.path.join(current_dir))

current_dir = os.path.normpath(current_dir)
sys.path.append(current_dir)

from Module.Components.config import get_file_path, Settings
from Module.Components.state_manager import StateManager
from Module.Common.scripts.llm.gemini_sdk import (
    types,
    get_safety_settings,
    format_content,
    get_content_config
)
from Module.Common.scripts.llm.utils.schema_response import ContentAnalyzer
from Module.Common.scripts.common.debug_utils import log_and_print

# 使用方式
settings = Settings(current_dir=current_dir)

gemini_client = genai.Client(api_key=settings.api_key)

MODEL_NAME = "gemini-2.0-flash-exp"


def gemini_generate_with_schema(
    client: Any,
    input_text: str,
    response_schema: Dict,
    system_prompt: str = ""
) -> Any:
    """使用Gemini生成带schema的响应

    Args:
        client: Gemini客户端实例
        input_text: 输入文本
        response_schema: 响应schema定义
        system_prompt: 系统提示词

    Returns:
        生成的响应内容
    """
    return client.models.generate_content(
        model=MODEL_NAME,
        contents=[format_content("user", input_text)],
        config=types.GenerateContentConfig(
            system_instruction=system_prompt,
            response_mime_type="application/json",
            response_schema=response_schema,
            safety_settings=get_safety_settings(),
        ),
    )


def game_response_formatter(response: Any) -> Dict[str, Any]:
    """格式化游戏响应

    Args:
        response: 原始响应内容

    Returns:
        格式化后的响应字典
    """
    if settings.config.get("log_level", "") == "debug":
        log_and_print("game_response_formatter:\n", response.text)
    updates = json.loads(response.text)

    state_updates = updates.get('stateUpdates', [])

    updates['stateUpdates'] = [u for u in state_updates if u is not None]

    return {
        'inventory': updates.get('itemUpdates', []),
        'character_state': updates.get('stateUpdates', [])
    }


def game_context_formatter(current_data: Dict[str, Any], content: str) -> str:
    """格式化游戏上下文

    Args:
        current_data: 当前游戏数据
        content: 内容文本

    Returns:
        格式化后的上下文字符串
    """
    # 获取配置
    state_config = settings.config["state_analysis"]
    extra_state_attribute = settings.config.get("extra_state_attributes", [])

    # 构建规则和提示
    rules = "\n".join(f"- {rule}" for rule in state_config["rules"])
    extra_state_hint = ""

    # 构建状态内容
    character_state = current_data.get('character_state', {})

    formatted_state = []
    for attr, state in character_state.items():
        if attr and state:
            formatted_state.append(f"{attr}: {state['state'] if state['state'] else '暂无'}")
            # formatted_state.append(f"{attr}: {state['state']}")
    final_state = "- " + '\n- '.join(formatted_state)

    state_content = f"""
故事发生前的状态
【物品清单】
{current_data.get('inventory', {})}

【角色状态】
{final_state}
"""
    # 构建完整内容
    formatted_content = state_config["template"].format(
        rules=rules,
        extra_hint=extra_state_hint
    ) + state_content

    # Debug日志
    if settings.config.get("log_level", "") == "debug":
        log_and_print("game_context_formatter:\n", formatted_content + "【最近故事内容】\n")

    # 返回带故事内容的完整格式
    return formatted_content + f"""
【最近故事内容】
{content}
【最近故事内容】
"""


def game_extra_response_formatter(response: Any) -> Dict[str, Any]:
    """格式化额外属性响应"""
    if settings.config.get("log_level", "") == "debug":
        log_and_print("game_extra_response_formatter:\n", response.text)
    updates = json.loads(response.text)
    state_updates = updates.get('stateUpdates', [])
    extra_state_attribute = settings.config.get("extra_state_attributes", [])[0]
    for update in state_updates:
        if 'new_value' in update:
            update[extra_state_attribute] = update.pop('new_value')
    return {'stateUpdates': state_updates}


def game_extra_context_formatter(base_result: Dict, current_data: Dict) -> str:
    """游戏特定的额外评估上下文格式化"""
    changes = []
    extra_attrs = settings.config.get('extra_state_attributes', [])
    attr_names = settings.config.get('state_attribute_names', {})
    guidance = settings.config.get('extra_value_evaluation', {}).get('guidance', [])

    for update in base_result.get('stateUpdates', []):
        attr = update['attribute']
        current_state = current_data.get('character_state', {}).get(attr, {})
        if update['to_state'] != current_state.get('state', ''):
            changes.extend([
                f"属性：{attr}",
                f"初始状态：{current_state.get('state', '')}",
                f"变化后的状态：{update['to_state']}"
            ])
            # 添加所有配置的extra属性
            for extra_attr in extra_attrs:
                attr_name = attr_names.get(extra_attr, extra_attr)
                changes.append(
                    f"初始{attr_name}值：{current_state.get(extra_attr, 0)}"
                )
            changes.append("")  # 添加空行分隔

    result = ""
    if changes:
        result = "\n".join(guidance) + "\n" + "\n".join(changes)

    if settings.config.get("log_level", "") == "debug":
        log_and_print("game_extra_context_formatter:\n", result, "\n id:", chat_data.get('current_id', 0))

    return result


def game_merge_updates(base_result: Dict, extra_result: Dict) -> Dict:
    """游戏特定的更新合并策略"""
    base_updates = base_result.get('character_state', [])
    extra_updates = extra_result.get('stateUpdates', [])

    # 记录每个属性首次出现的位置和更新
    seen_attrs = {}
    final_updates = []

    # 检查是否有extra属性用于比较
    extra_state_attributes = settings.config.get("extra_state_attributes", [])
    extra_state_attribute = extra_state_attributes[0] if extra_state_attributes else None

    # 处理基础更新
    for update in base_updates:
        attr = update['attribute']
        # 标准化更新数据格式
        normalized_update = {
            'attribute': attr,
            'from_state': update.get('from_state'),
            'state': update.get('to_state'),  # 统一使用 state 作为键名
            **{k: v for k, v in update.items() if k not in ['attribute', 'from_state', 'to_state', 'state']}
        }

        if attr in seen_attrs:
            prev_idx = seen_attrs[attr]
            prev_update = final_updates[prev_idx]

            if (extra_state_attribute in update and
                extra_state_attribute in prev_update):
                # 如果新的extra值更大，则替换旧的更新
                if update[extra_state_attribute] > prev_update[extra_state_attribute]:
                    final_updates[prev_idx] = normalized_update
            else:
                # 没有extra属性时保留后出现的更新
                final_updates[prev_idx] = normalized_update
        else:
            seen_attrs[attr] = len(final_updates)
            final_updates.append(normalized_update)

    # 处理额外属性更新
    for extra in extra_updates:
        attr = extra['attribute']
        if attr in seen_attrs:
            idx = seen_attrs[attr]
            # 将额外属性合并到对应的更新中
            final_updates[idx].update({
                k: v for k, v in extra.items()
                if k != 'attribute'
            })

    # 构建最终结果
    result = {k: v for k, v in base_result.items() if k != 'character_state'}
    result['character_state'] = final_updates
    result['extraUpdates'] = extra_result

    return result


# 创建分析器实例
analyzer = ContentAnalyzer(
    llm_client=gemini_client,
    generate_func=gemini_generate_with_schema,
    response_schema=settings.response_schema,
    system_prompt=settings.state_system_prompt,
    context_formatter=game_context_formatter,
    response_formatter=game_response_formatter,
    # 额外评估配置
    extra_schema=settings.config.get('extra_value_evaluation', {}).get('schema'),
    extra_prompt=settings.config.get('extra_value_evaluation', {}).get('system_role'),
    extra_context_formatter=game_extra_context_formatter,
    extra_response_formatter=game_extra_response_formatter,
    merge_updates=game_merge_updates
)


def get_gradio_version() -> str:
    """获取Gradio版本号

    Returns:
        Gradio版本号字符串
    """
    return gr.__version__


def create_initial_state() -> Dict:
    """创建初始游戏状态

    Returns:
        包含初始状态的字典
    """
    initial_state = {
        "gr_version": get_gradio_version(),
        "story_chapter": "起因",
        "story_chapter_stage": 1,
        "inventory": {},
        "character_state": {}
    }
    # 深度拷贝确保完全隔离
    for attr in settings.config["state_attributes"]:
        initial_state["character_state"][attr] = {
            "state": "",  # 主状态
        }
        # 添加额外属性
        for extra_attr in settings.config.get("extra_state_attributes", []):
            initial_state["character_state"][attr][extra_attr] = -1
    return initial_state


game_state = create_initial_state()

state_manager = StateManager(create_initial_state(), settings.config)

# 区分用于处理的历史记录和用于显示的历史记录
# 使用字典来存储对话历史和当前ID
chat_data = {
    "current_id": 0,
    "history": []  # 列表中存储对话记录字典，每条记录包含role、content、only_for_display和id属性
}


def detect_state_changes(game_state_dict: dict, story_output: str) -> Dict:
    """检测游戏状态变化

    Args:
        game_state_dict: 当前游戏状态字典
        story_output: 故事输出文本

    Returns:
        状态更新信息
    """
    updates = analyzer.analyze(game_state_dict, story_output)
    return state_manager.apply_updates(updates)


def _handle_special_messages(message: str, history: List[Tuple[str, str]], ignore_job: str) -> str:
    """处理特殊消息命令"""
    jobs_config = settings.config.get("explored_jobs", {})

    def apply_jobs_template(template_text: str) -> str:
        """应用职业排除模板"""
        if message == jobs_config.get("trigger") and '{explored_jobs}' in template_text:
            explored_text = jobs_config["template"].format(jobs=ignore_job) if ignore_job else ""
            return template_text.format(explored_jobs=explored_text)
        return template_text

    if message == "开始" and not history:
        begin_message = settings.begin
        if settings.config["initial_state"]:
            begin_message += (
                "\n请在应当确认随机内容的时机一并初始化状态和持有物品，状态属性清单如下：\n" +
                "\n".join(settings.config["state_attributes"])
            )
        return apply_jobs_template(begin_message)

    if message == "确认" and len(history) == 2:
        return apply_jobs_template(settings.confirm)

    return message


def _should_append_state(message: str) -> Tuple[bool, bool, str]:
    """判断是否需要附加状态信息

    Args:
        message: 输入的消息

    Returns:
        Tuple[bool, bool, str]:
        - 是否需要附加状态
        - 是否是控制命令
        - 处理后的消息
    """
    # 检查是否是控制命令
    is_control = message.startswith('ct')
    processed_message = message[2:].strip() if is_control else message

    # 判断是否需要附加状态（控制命令一定不附加，其他情况按原有逻辑判断）
    should_append = False if is_control else (
        game_state["story_chapter_stage"] > 1 or
        game_state["story_chapter"] != "起因" or
        chat_data["current_id"] > 1
    )

    return should_append, is_control, processed_message


def _process_response(chunk: Any, response: str) -> Tuple[str, bool]:
    """处理响应块，返回更新的响应和是否需要中断"""
    if not chunk.text:
        return response, False

    if "状态变化：" in chunk.text:
        response += chunk.text.split("状态变化：")[0]
        return response, True

    if ("【情节完成】" in chunk.text) and (chat_data["current_id"] > 1):
        response += chunk.text.split("【情节完成】")[0] + "【情节完成】"
        return response, True

    return response + chunk.text, False


def build_contents(message=None, before_message=None):
    """构建内容列表"""
    contents = []
    for val in chat_data["history"]:
        if val.get("only_for_display", False):
            continue
        contents.append(format_content(
            val["role"],
            val["content"]
        ))

    if before_message:
        contents.append(format_content(
            "assistant",
            before_message
        ))

    if message:
        contents.append(format_content(
            "user",
            message
        ))
    return contents


def respond(
    message: str,
    history: List[Tuple[str, str]],
    use_system_message: bool,
    add_extra_message: bool,
    auto_analysis_state: bool,
    ignore_job: str,
) -> str:
    """处理用户输入并生成响应

    Args:
        message: 用户输入消息
        history: 对话历史
        use_system_message: 是否使用系统消息
        ignore_job: 要忽略的职业

    Returns:
        生成的响应文本
    """
    # 构建对话历史
    message = _handle_special_messages(message, history, ignore_job)
    # 判断是否需要附加状态信息

    should_append, is_control, message = _should_append_state(message)

    if (should_append) and (add_extra_message):
        message += state_manager.get_state_str()

    if message:
        # 处理普通消息
        contents = build_contents(message)
        response = ""
        config = get_content_config(use_system_message, settings.system_role)
        chat_data["current_id"] += 1
        chat_data["history"].append({
            "role": "user",
            "content": message,
            "idx": chat_data["current_id"]
        })
        if settings.config.get("log_level", "") in ["debug", "info"]:
            separator = "\n" + "★"*30 + "《CONTENT START》" + "★"*30 + "\n"
            separator_end = "\n" + "☆"*30 + "《CONTENT END》" + "☆"*30 + "\n"
            log_and_print("content before main response:\n", separator, message, separator_end)

        for chunk in gemini_client.models.generate_content_stream(
            model=MODEL_NAME,
            contents=contents,
            config=config
        ):
            response, should_break = _process_response(chunk, response)
            yield response
            if should_break:
                break

        if settings.config.get("log_level", "") in ["debug", "info"]:
            log_and_print("after main response:\n", response)

        chat_data["history"].append({
            "role": "assistant",
            "content": response,
            "idx": chat_data["current_id"]
        })

        if not is_control and auto_analysis_state:
            updates_str, _ = detect_state_changes(state_manager.get_state(), response)
            if updates_str:
                chat_data["history"].append({
                    "role": "assistant",
                    "content": updates_str,
                    "only_for_display": True,
                    "idx": chat_data["current_id"]
                })
                if "状态变化：" in response:
                    yield response + "\n" + updates_str
                else:
                    yield response + "\n状态变化：\n" + updates_str


with gr.Blocks(theme="soft") as demo:
    # 1. 创建界面组件
    chatbot = gr.ChatInterface(
        respond,
        title=settings.config["title"],
        type="messages",
        # examples=[["开始",True,settings.config.get("explored_jobs", {}).get("default", "")]],
        chatbot=gr.Chatbot(
            placeholder="输入 【开始】 开始进行创作",
            height="80vh",
            # show_share_button=True,
            editable="user",
            show_copy_all_button=True,
            type="messages",
        ),
        additional_inputs=[
            # gr.Row([
            gr.Checkbox(value=True, label="Use system message"),
            gr.Checkbox(value=True, label="Add Extra message"),
            gr.Checkbox(value=True, label="Auto analysis state"),
            # ]),
            gr.Textbox(
                value=settings.config.get("explored_jobs", {}).get("default", ""),
                label="ignore job"
            ),
        ],
    )

    # 2. 创建状态显示组件
    outputs = []
    with gr.Accordion("查看故事状态", open=False):
        state_output = gr.JSON(value=state_manager.get_state())
        outputs.append(state_output)

    if settings.config["show_chat_history"]:
        with gr.Accordion("查看历史对话", open=False):
            history_output = gr.JSON(value=chat_data["history"])
            outputs.append(history_output)

    # 3. 定义事件处理函数（保持全局状态访问）
    def update_state() -> List[Any]:
        return ([state_manager.get_state(), chat_data["history"]]
                if settings.config["show_chat_history"]
                else state_manager.get_state())

    def clear_state() -> List[Any]:
        chat_data['current_id'] = 0
        chat_data['history'] = []
        state_manager.state = create_initial_state()
        state_manager.state_history = create_initial_state()
        return update_state()

    def undo_state() -> List[Any]:
        if chat_data["current_id"] > 0:
            chat_data["history"] = [
                msg for msg in chat_data["history"]
                if msg["idx"] != chat_data["current_id"]
            ]
            chat_data["current_id"] -= 1
        state_manager.reset_state()
        return update_state()

    # 4. 绑定事件处理
    chatbot.chatbot.change(fn=update_state, outputs=outputs)
    chatbot.chatbot.clear(fn=clear_state, outputs=outputs)
    chatbot.chatbot.undo(fn=undo_state, outputs=outputs)
    chatbot.chatbot.retry(fn=undo_state, outputs=outputs)


if __name__ == "__main__":
    import asyncio
    # 在程序启动前添加以下代码
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
    try:
        # 尝试获取命令行参数
        import argparse
        parser = argparse.ArgumentParser()
        parser.add_argument("--host", default="0.0.0.0", help="Host to bind to")
        parser.add_argument("--port", type=int, default=7860, help="Port to bind to")
        parser.add_argument("--share", action="store_true", help="Enable sharing")
        args = parser.parse_args()
        launch_kwargs = {
            "server_name": args.host,
            "server_port": args.port,
            "share": args.share
        }
    except:
        # 在notebook中运行时使用默认值
        launch_kwargs = {
            "server_name": "0.0.0.0",
            "share": False,
            "debug": True
        }

    ssl_cert = get_file_path("localhost+1.pem", current_dir=current_dir)
    ssl_key = get_file_path("localhost+1-key.pem", current_dir=current_dir)

    if os.path.exists(ssl_cert) and os.path.exists(ssl_key):
        launch_kwargs.update({
            "ssl_certfile": ssl_cert,
            "ssl_keyfile": ssl_key
        })

    # 启动应用
    demo.launch(**launch_kwargs)


In [ ]:

# ===== 1. 导入依赖 =====
# 标准库导入
import os
import sys
import json
from dotenv import load_dotenv
from IPython.display import display, Markdown


# ===== 2. 初始化配置 =====
# 获取当前文件所在目录的绝对路径
if "__file__" in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
    root_dir = os.path.normpath(os.path.join(current_dir, ".."))
else:
    # 在 Jupyter Notebook 环境中
    current_dir = os.getcwd()
    current_dir = os.path.join(current_dir, "..")
    root_dir = os.path.normpath(os.path.join(current_dir))

current_dir = os.path.normpath(current_dir)
sys.path.append(current_dir)

from Module.Components.config import Settings, load_prompt_file
from Module.Common.scripts.llm.gemini_sdk import types, get_safety_settings
from google import genai

settings = Settings(current_dir=current_dir)

# Load API keys
load_dotenv(os.path.join(current_dir, ".env"))
api_key = os.getenv("GEMINI_API_KEY")
gemini_client = genai.Client(api_key=api_key)


model_name="gemini-2.0-flash-thinking-exp-01-21"
# 创建分析器实例
mode = "simple"

if mode == "extra":
    system_role = settings.config.get('extra_value_evaluation', {}).get('system_role')
    new_schema = settings.config.get('extra_value_evaluation', {}).get('schema')
elif mode == "state":
    system_role = settings.state_system_prompt
    new_schema = settings.response_schema
elif mode == "simple":
    system_role = load_prompt_file("system_role_trainer.md", current_dir=current_dir)
    system_role = "你是一个睿智的吟游诗人"
    new_schema = None

t1=get_safety_settings()
t1[4].threshold="BLOCK_ONLY_HIGH"
# t1

chat = gemini_client.chats.create(model=model_name,
    config=types.GenerateContentConfig(
        system_instruction=system_role,
        response_mime_type="application/json" if new_schema else "text/plain",
        response_schema=new_schema,
        temperature=0.7,
        safety_settings=t1
    ),)

context="""
hello
"""

message_list=["""
为了方便阅读，我拆分排版一下。
这是一开始的初始提示词，由<>包括：
<初始提示词开始>
""",settings.begin,
"""
<初始提示词结束>
这是要分析的内容
"""]
# """,context]
message="\n".join(message_list)

message="你好呀"

response = chat.send_message(message)

print(api_key)

display(Markdown(response.text))


In [ ]:
import requests
from playsound import playsound  # 用于播放音频

# 配置参数（替换为你的实际参数）
COZE_API_BASE = "https://api.coze.cn"
BOT_ID = "7316745484305989638"  # 替换为你的 Bot ID
ACCESS_TOKEN = "pat_mmmXVkHLMwTSqaoypfFyd213faLOV1MAq7rzNNRFOMnhW0DpGa4H9pWZzGPS4uwM"  # 替换为你的访问令牌
VOICE_ID = "peach"  # 中文女声音色

def coze_tts(text: str, output_file: str = "output.mp3") -> bool:
    """
    调用 Coze TTS 生成语音
    :param text: 需要合成的文本
    :param output_file: 输出音频文件路径（支持 .mp3/.wav）
    :return: 是否成功
    """
    url = f"{COZE_API_BASE}/open_api/v2/tts"
    headers = {
        "Authorization": f"Bearer {ACCESS_TOKEN}",
        "Content-Type": "application/json",
        "Accept": "application/json"
    }
    payload = {
        "bot_id": BOT_ID,
        "text": text,
        "voice_id": VOICE_ID,
        "model": "general",  # 默认模型
        "format": "mp3"  # 可选 mp3/wav
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()  # 检查 HTTP 错误

        # 保存音频文件
        with open(output_file, "wb") as f:
            f.write(response.content)
        return True
    except requests.exceptions.RequestException as e:
        print(f"请求失败: {e}")
        return False

# 使用示例
if __name__ == "__main__":
    input_text = "你好，欢迎使用 Coze 语音合成服务！"

    # 生成语音文件
    if coze_tts(input_text, "speech.mp3"):
        print("语音生成成功，正在播放...")
        # 播放音频（需安装 playsound 库）
        playsound("speech.mp3")
    else:
        print("语音生成失败")

In [ ]:
import requests
from IPython.display import Audio

def local_tts(text: str, speaker: str = "步非烟", instruct: str = "性感诱惑的", output_file: str = "output.mp3") -> bool:
    """
    调用本地 TTS 服务生成语音
    :param text: 需要合成的文本
    :param speaker: 说话人
    :param instruct: 语气指令
    :param output_file: 输出音频文件路径
    :return: 是否成功
    """
    url = f"http://localhost:9880/"
    params = {
        "text": text,
        "speaker": speaker,
        "instruct": instruct
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # 检查 HTTP 错误

        # 保存音频文件
        with open(output_file, "wb") as f:
            f.write(response.content)
        return True
    except requests.exceptions.RequestException as e:
        print(f"请求失败: {e}")
        return False

input_text = """
hello
"""
local_tts(input_text)
Audio("output.mp3", autoplay=True)

In [ ]:
import edge_tts
from IPython.display import Audio

voice_language="zh-CN"
# voice_language="zh-TW"
voice_name="HsiaoYuNeural"
voice_name="XiaoxiaoNeural"

voice=f"{voice_language}-{voice_name}"

async def tts_edge(text):
    communicate = edge_tts.Communicate(text, voice=voice)
    await communicate.save("output1.mp3")
    return "output1.mp3"

# 生成并播放音频
text="""
hello
"""
audio_path = await tts_edge(text)
Audio(audio_path, autoplay=True)  # 自动播放

In [ ]:
def chunk(text):
    # Split text into chunks based on user: and assistant: markers
    chunks = []
    current_chunk = ""
    current_speaker = None

    lines = text.split('\n')
    for line in lines:
        if line.startswith('user:'):
            if current_chunk:
                chunks.append((current_speaker, current_chunk.strip()))
            current_speaker = 'user'
            current_chunk = line[5:] # Remove 'user:'
        elif line.startswith('assistant:'):
            if current_chunk:
                chunks.append((current_speaker, current_chunk.strip()))
            current_speaker = 'assistant'
            current_chunk = line[10:] # Remove 'assistant:'
        else:
            current_chunk += '\n' + line

    # Add final chunk
    if current_chunk:
        chunks.append((current_speaker, current_chunk.strip()))

    # Extract only user messages
    return chunks

with open(os.path.join(current_dir, "local_setting/story.md"), "r", encoding="utf-8") as f:
    story_content = f.read()

user_messages = [chunk[1] for chunk in chunk(story_content) if chunk[0] == 'user']
user_messages


In [ ]:
new_talk = """
很棒，继续吧！
"""
result = chat.send_message(new_talk)
display(Markdown(result.text))

In [ ]:
from google import genai
from google.genai import types

client = genai.Client(api_key=api_key)

response = client.models.generate_content(
    model="gemini-2.0-flash-exp", contents="What is your name?"
)
response

In [ ]:
def get_current_weather(location: str) -> str:
    """Returns the current weather.

    Args:
      location: The city and state, e.g. San Francisco, CA
    """
    return "sunny"


response = client.models.generate_content(
    model="gemini-2.0-flash-exp",
    contents="What is the weather like in Boston?",
    config=types.GenerateContentConfig(tools=[get_current_weather]),
)

print(response.text)

In [ ]:
from pydantic import BaseModel


class CountryInfo(BaseModel):
    name: str
    population: int
    capital: str
    continent: str
    gdp: int
    official_language: str
    total_area_sq_mi: int


response = client.models.generate_content(
    model="gemini-2.0-flash-exp",
    contents="Give me information for the United States.",
    config=types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=CountryInfo,
    ),
)
print(response.text)

In [ ]:
for chunk in client.models.generate_content_stream(
    model="gemini-2.0-flash-exp", contents="Tell me a story in 300 words."
):
    print(chunk.text, end="")

In [ ]:
async for response in client.aio.models.generate_content_stream(
    model="gemini-2.0-flash-exp", contents="Tell me a story in 300 words."
):
    print(response.text, end="")

In [ ]:
response = client.models.embed_content(
    model="text-embedding-004",
    contents="What is your name?",
)
print(response)

In [ ]:
# Generate Image
image_prompt ="""
1. Top: A white lace splicing long-sleeved dress. It reaches the knees, with a high neckline design that outlines a slender and elegant neckline. Lace and white asylum are in line with the freshness and innocence of girls, but also reveal a hint of sexiness.
2. Skirt: Outside choose a white lace peplum dress, with a circle of lace peplum decoration at the knees, which looks romantic and lovely. Inside is a black gauze suspender skirt, vaguely showing tight curves and stocking edges. This contrasting design makes it difficult to grasp the truth inside.
3. Underwear: A French lace bustier that perfectly outlines the full breasts and half conceals them. Seemingly harmless on the surface, it is extremely sexy in reality, forming a sharp contrast with the outer jacket.
4. Shoes: A pair of white thick-soled lace-up sandals, as lovely as a girl. Inside are a pair of black stockings and fine high-heeled ankle locks that can be vaguely seen under the skirt from time to time, proclaiming the real unrestrained side.
5. Hairstyle and makeup: Innocent medium-length shaggy hair and light makeup. But the lips are stained with extremely red lipstick, which is fascinating. This contrast becomes an important breakthrough in the overall styling, allowing people to catch a glimpse of the real look at a glance.

On the surface, this styling still reflects the innocent and lovely girl's sense. But in the details, it reveals traces of unrestrained and sexy everywhere, making it impossible to ignore the existence of the real look. It creates a strong contrast between the two styles, but also integrates them into one, making it difficult for people to clearly judge for a while. This is precisely the effect and highest realm that sissy bimbo has always pursued.
This styling makes her an seemingly innocent but extremely charming existence. She walks between the inside and outside, switching back and forth between two completely different worlds, fascinating everyone but also incomprehensible. This is what she pursues, and it is also the state she most desires to achieve in her life. She is an indefinable woman, a perfect representative who truly achieves inner and outer cultivation.
"""

response1 = client.models.generate_image(
    model="imagen-3.0-generate-001",
    prompt=image_prompt,
    config=types.GenerateImageConfig(
        # negative_prompt="human",
        number_of_images=1,
        include_rai_reason=True,
        output_mime_type="image/jpeg",
        person_generation="ALLOW_ALL",
        safety_filter_level="BLOCK_NONE"
    ),
)
response1.generated_images[0].image.show()

In [ ]:
response = chat.send_message(
"""
故事先到这里吧。接下来我需要你协助做另一个事情。
我计划把这个创作过程修改成一个由llm驱动的游戏世界，故事的大纲仍然如此，但职业和发生的事情每次都是随机生成的。
此外，在文字冒险的过程中，玩家在每个阶段都需要经过最多三四步的操作，然后触发这个阶段的标志性事件，然后再过度到下一个阶段。

为了实现这个效果，我需要先完成各个层级的系统提示词。除了参考模板里的字段和特殊的专有名词，其他就都用中文。
请你结合我给你的模板的思路，和我们发生的故事、大纲以及要随机的部分，精心设计一个属于这个故事的数据结构，并帮我完成这些系统提示词
让我们一步一步来，先沟通清楚推荐的数据结构。

参考模板：
<system_role_prompt>
Your job is to help create interesting fantasy worlds that \
players would love to play in.
Instructions:
- Only generate in plain text without formatting.
- Use simple clear language without being flowery.
- You must stay below 3-5 sentences for each description.

<world_prompt>
Generate a creative description for a unique fantasy world with an
interesting concept around cities build on the backs of massive beasts.

Output content in the form:
World Name: <WORLD NAME>
World Description: <WORLD DESCRIPTION>

World Name:

<kingdom_prompt>
Create 3 different kingdoms for a fantasy world.
For each kingdom generate a description based on the world it's in. \
Describe important leaders, cultures, history of the kingdom.\

Output content in the form:
Kingdom 1 Name: <KINGDOM NAME>
Kingdom 1 Description: <KINGDOM DESCRIPTION>
Kingdom 2 Name: <KINGDOM NAME>
Kingdom 2 Description: <KINGDOM DESCRIPTION>
Kingdom 3 Name: <KINGDOM NAME>
Kingdom 3 Description: <KINGDOM DESCRIPTION>

World Name: {world['name']}
World Description: {world['description']}

Kingdom 1

def get_town_prompt(world, kingdom):
    return f"
    Create 3 different towns for a fantasy kingdom abd world. \
    Describe the region it's in, important places of the town, \
    and interesting history about it. \

    Output content in the form:
    Town 1 Name: <TOWN NAME>
    Town 1 Description: <TOWN DESCRIPTION>
    Town 2 Name: <TOWN NAME>
    Town 2 Description: <TOWN DESCRIPTION>
    Town 3 Name: <TOWN NAME>
    Town 3 Description: <TOWN DESCRIPTION>

    World Name: {world['name']}
    World Description: {world['description']}

    Kingdom Name: {kingdom['name']}
    Kingdom Description {kingdom['description']}

    Town 1 Name:"

def get_npc_prompt(world, kingdom, town):
    return f"
    Create 3 different characters based on the world, kingdom \
    and town they're in. Describe the character's appearance and \
    profession, as well as their deeper pains and desires. \

    Output content in the form:
    Character 1 Name: <CHARACTER NAME>
    Character 1 Description: <CHARACTER DESCRIPTION>
    Character 2 Name: <CHARACTER NAME>
    Character 2 Description: <CHARACTER DESCRIPTION>
    Character 3 Name: <CHARACTER NAME>
    Character 3 Description: <CHARACTER DESCRIPTION>

    World Name: {world['name']}
    World Description: {world['description']}

    Kingdom Name: {kingdom['name']}
    Kingdom Description: {kingdom['description']}

    Town Name: {town['name']}
    Town Description: {town['description']}

    Character 1 Name:"

<item manager>
You are an AI Game Assistant. \
Your job is to detect changes to a player's \
inventory based on the most recent story and game state.
If a player picks up, or gains an item add it to the inventory \
with a positive change_amount.
If a player loses an item remove it from their inventory \
with a negative change_amount.
Given a player name, inventory and story, return a list of json update
of the player's inventory in the following form.
Only take items that it's clear the player (you) lost.
Only give items that it's clear the player gained.
Don't make any other item updates.
If no items were changed return {"itemUpdates": []}
and nothing else.

Response must be in Valid JSON
Don't add items that were already added in the inventory

Inventory Updates:
{
    "itemUpdates": [
        {"name": <ITEM NAME>,
        "change_amount": <CHANGE AMOUNT>}...
    ]
}

""")
display(Markdown(response.text))
response